In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

In [3]:
import pandas as pd

In [6]:
df = pd.read_csv('../csv/spam_mail.csv')
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [9]:
df.groupby('Category').describe()

Message                                                            \
           count unique                                                top   
Category                                                                     
ham         4825   4516                             Sorry, I'll call later   
spam         747    641  Please call our customer service representativ...   

               
         freq  
Category       
ham        30  
spam        4

In [11]:
df['spam'] = df.Category.apply(lambda x:1 if x=='spam' else 0)
df.head()

,Category,Message,spam
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0


In [12]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df.Message,df.spam, random_state=43)

In [15]:
len(X_train)

4179

In [16]:
bert_preprocess=  hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')


In [17]:
def get_sentences_embedding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']

In [29]:
e= get_sentences_embedding([
    'tomato',
    'chili',
    'Neeraj',
    'Girja'
])

In [20]:
from sklearn.metrics.pairwise import cosine_similarity

In [31]:
cosine_similarity([e[0]],[e[3]])

array([[0.9885458]], dtype=float32)

In [33]:
text_input = tf.keras.layers.Input(shape=(),dtype=tf.string, name='text')

In [40]:
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

l= tf.keras.layers.Dropout(0.3, name="dropout")(outputs['pooled_output'])

In [41]:
l= tf.keras.layers.Dense(1,activation='sigmoid', name='output')(l)

In [44]:
model = tf.keras.Model(inputs=[text_input], outputs=[l])

In [45]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                    

In [46]:
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
model.fit(X_train,y_train,epochs=5)

Epoch 1/5
 47/131 [=========>....................] - ETA: 17:24 - loss: 0.4175 - accuracy: 0.8531